# 임포트

In [1]:
import numpy as np
import pandas as pd
import os

# 데이터 확인

In [2]:
data = pd.read_csv('airbnbmark1.csv')
data.shape

C:\Users\shkim\AppData\Local\Temp\ipykernel_29044\885709729.py:1: DtypeWarning: Columns (27,61,62,94,95) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('airbnbmark1.csv')


(153254, 106)

In [3]:
data.head(3)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2595,https://www.airbnb.com/rooms/2595,20200212052319,2020-02-12,Skylit Midtown Castle,"Beautiful, spacious skylit studio in the heart...","- Spacious (500+ft²), immaculate and nicely fu...","Beautiful, spacious skylit studio in the heart...",none,Centrally located in the heart of Manhattan ju...,...,f,f,strict_14_with_grace_period,t,t,2,2,0,0,0.39
1,3831,https://www.airbnb.com/rooms/3831,20200212052319,2020-02-13,Cozy Entire Floor of Brownstone,Urban retreat: enjoy 500 s.f. floor in 1899 br...,Greetings! We own a double-duplex brownst...,Urban retreat: enjoy 500 s.f. floor in 1899 br...,none,Just the right mix of urban center and local n...,...,f,f,moderate,f,f,1,1,0,0,4.69
2,5099,https://www.airbnb.com/rooms/5099,20200212052319,2020-02-12,Large Cozy 1 BR Apartment In Midtown East,My large 1 bedroom apartment has a true New Yo...,I have a large 1 bedroom apartment centrally l...,My large 1 bedroom apartment has a true New Yo...,none,My neighborhood in Midtown East is called Murr...,...,f,f,moderate,t,t,1,1,0,0,0.59


In [4]:
data.describe()
# 아래 정보들은 object dtype을 가진 데이터는 포함하지 않으므로 주의
# 결측치가 있음을 확인 가능

,id,scrape_id,thumbnail_url,medium_url,xl_picture_url,host_id,host_listings_count,host_total_listings_count,latitude,longitude,...,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
count,1.532540e+05,1.532540e+05,0.0,0.0,0.0,1.532540e+05,153050.000000,153050.000000,153254.000000,153254.000000,...,119312.000000,119231.000000,119279.000000,119222.000000,119222.000000,153254.000000,153254.00000,153254.000000,153254.000000,122289.000000
mean,2.225193e+07,2.020021e+13,NaN,NaN,NaN,8.297160e+07,21.179536,21.179536,40.729086,-73.950828,...,9.279025,9.736109,9.742310,9.593817,9.383830,7.609159,5.85804,1.515921,0.137295,1.336353
std,1.293543e+07,8.583871e+07,NaN,NaN,NaN,9.537811e+07,146.649053,146.649053,0.054865,0.047666,...,1.083864,0.744320,0.760339,0.749935,0.934644,32.970765,32.41745,6.984752,1.287762,1.648065
min,2.060000e+03,2.020010e+13,NaN,NaN,NaN,2.259000e+03,0.000000,0.000000,40.499790,-74.244420,...,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,0.00000,0.000000,0.000000,0.000000
25%,1.065982e+07,2.020010e+13,NaN,NaN,NaN,9.368118e+06,1.000000,1.000000,40.689700,-73.983160,...,9.000000,10.000000,10.000000,9.000000,9.000000,1.000000,0.00000,0.000000,0.000000,0.190000
50%,2.203224e+07,2.020021e+13,NaN,NaN,NaN,3.763366e+07,1.000000,1.000000,40.723360,-73.955150,...,10.000000,10.000000,10.000000,10.000000,10.000000,1.000000,1.00000,0.000000,0.000000,0.690000
75%,3.432368e+07,2.020031e+13,NaN,NaN,NaN,1.365562e+08,2.000000,2.000000,40.763090,-73.934010,...,10.000000,10.000000,10.000000,10.000000,10.000000,2.000000,1.00000,1.000000,0.000000,2.000000
max,4.289272e+07,2.020031e+13,NaN,NaN,NaN,3.414399e+08,2345.000000,2345.000000,40.914290,-73.708400,...,10.000000,10.000000,10.000000,10.000000,10.000000,302.000000,302.00000,133.000000,34.000000,66.360000


In [5]:
# DtypeWarning: Columns (27,61,62,94,95) have mixed types.
# Specify dtype option on import or set low_memory=False.
data.iloc[:, [27,61,62,94,95]]

,host_acceptance_rate,weekly_price,monthly_price,license,jurisdiction_names
0,38%,"$1,995.00",NaN,NaN,NaN
1,96%,$575.00,"$2,100.00",NaN,NaN
2,71%,NaN,NaN,NaN,NaN
3,67%,NaN,NaN,NaN,NaN
4,100%,$470.00,NaN,NaN,NaN
...,...,...,...,...,...
153249,100%,NaN,NaN,NaN,NaN
153250,99%,NaN,NaN,NaN,NaN
153251,NaN,NaN,NaN,NaN,NaN
153252,100%,NaN,NaN,NaN,NaN


In [6]:
data[data.license.isna() == False].license

26279     35-2593481
29458     81-1092926
29459     81-1092926
29460     81-1092926
29461     81-1092926
             ...    
152960    46-4091213
152961    46-4091213
152962    46-4091213
152965    46-4091213
152966    46-4091213
Name: license, Length: 86, dtype: object

# 전처리

## 분석

### 전처리 또는 drop이 필요한 것들


* 하위 항목 분리 필요
  * host_verification
  * amenities
* classify 필요
  * cancellation_pollicy
  
* 종속성 확인(street)
  * street
  * neightborhood
  * neighborhood_cleansed
  * neighborhood_group
  * city
  * state
  * zipcode
  * market
  * smart_location 종속성 확인

* $ 표기 제거
  * price
  * weekly_price
  * monthly_price
  * security_deposit
  * cleaning_fee
  * extra_people
* % 표기 제거
  * host_acceptance_rate
* 적합성 판단 필요
  * calendar_updated
  * calendar_last_scraped
  * is_location_exact 수상한 컬럼

* 날자 표기 통일
  * first_review
  * last_review

### drop한 것들과 그 사유


* 모든 값이 동일
  * experience_offered; 모든 값이 null
  * country; US
  * country_code; US
  * requires_license; f
  * license; 86개 제외하고 모두 null
  * jurisdiction_names; 53개 제외하고 모두 null
  * is_business_travel_ready; f
* 얻을 정보 없음
  * scrape_id
  * last_scraped; scrape 관련 메타정보, 얻을 정보가 없음.
  * url
    * listing_url
    * airbnb_url
    * thumbnail_url
    * medium_url
    * picture_url
    * xl_pictuere_url
    * host_thumbnail_url
    * host_picture_url

## Drop

In [7]:
column_to_drop = [
    'listing_url',
    'scrape_id',
    'last_scraped',
    'experiences_offered',
    'thumbnail_url',
    'medium_url',
    'picture_url',
    'xl_picture_url',
    'host_thumbnail_url',
    'host_picture_url',
    'country',
    'country_code',
    'requires_license',
    'license',
    'jurisdiction_names',
    'is_business_travel_ready']
for column in column_to_drop:
    try:
        data = data.drop(column, axis='columns')
    except KeyError as e:
        print(e)

## 데이터 결측치/타입/오류 확인 및 제거

### 데이터 결측치 확인

In [8]:
count = len(data.id)
df_na_count = data.isna().sum()
df_na_rate = df_na_count / count
df_na = pd.concat([df_na_rate, df_na_count], axis=1)
df_na.columns = ['rate', 'count_na']
df_na[df_na.count_na != 0].sort_values(by='rate', ascending=False)

,rate,count_na
square_feet,0.992490,152103
monthly_price,0.906808,138972
weekly_price,0.892655,136803
notes,0.594764,91150
host_acceptance_rate,0.517898,79370
access,0.470128,72049
interaction,0.403676,61865
host_about,0.399050,61156
house_rules,0.389393,59676
host_response_rate,0.361746,55439


### 데이터 중복 확인

In [9]:
print("Dataframe 크기:", data.shape)
for column in data.columns:
    count = len(data.id)
    overlap_rate = count / len(data[column].unique())
    print(f"평균적으로 {column}의 unique한 값이 {len(data.id)/len(data.id.unique()):.05f}회 나타남")

print("\n\n==> 동일한 airbnb를 평균 2.67091회 스크래핑하는 듯. 숙소가 같으면 row가 항상 같은가? 확인하기.")

Dataframe 크기: (153254, 90)
평균적으로 id의 unique한 값이 2.67091회 나타남
평균적으로 name의 unique한 값이 2.67091회 나타남
평균적으로 summary의 unique한 값이 2.67091회 나타남
평균적으로 space의 unique한 값이 2.67091회 나타남
평균적으로 description의 unique한 값이 2.67091회 나타남
평균적으로 neighborhood_overview의 unique한 값이 2.67091회 나타남
평균적으로 notes의 unique한 값이 2.67091회 나타남
평균적으로 transit의 unique한 값이 2.67091회 나타남
평균적으로 access의 unique한 값이 2.67091회 나타남
평균적으로 interaction의 unique한 값이 2.67091회 나타남
평균적으로 house_rules의 unique한 값이 2.67091회 나타남
평균적으로 host_id의 unique한 값이 2.67091회 나타남
평균적으로 host_url의 unique한 값이 2.67091회 나타남
평균적으로 host_name의 unique한 값이 2.67091회 나타남
평균적으로 host_since의 unique한 값이 2.67091회 나타남
평균적으로 host_location의 unique한 값이 2.67091회 나타남
평균적으로 host_about의 unique한 값이 2.67091회 나타남
평균적으로 host_response_time의 unique한 값이 2.67091회 나타남
평균적으로 host_response_rate의 unique한 값이 2.67091회 나타남
평균적으로 host_acceptance_rate의 unique한 값이 2.67091회 나타남
평균적으로 host_is_superhost의 unique한 값이 2.67091회 나타남
평균적으로 host_neighbourhood의 unique한 값이 2.67091회 나타남
평균적으로 host_listings_count의 uniq

### Price류 $ 표기 제거, 숫자형으로 변환

In [10]:
data.last_review

0         2019-11-04
1         2020-02-08
2         2019-10-13
3         2019-12-02
4         2020-01-28
             ...    
153249           NaN
153250           NaN
153251           NaN
153252           NaN
153253           NaN
Name: last_review, Length: 153254, dtype: object